In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

result = load_dotenv(find_dotenv(), override=True)

if result:
    print("Variables de entorno cargadas exitosamente.")
else:
    print("No se pudo cargar el archivo .env.")

    # Crear el archivo .env con las variables deseadas si no existe
    with open(".env", "w") as f:
        f.write(f"OPENAI_API_KEY=MY_OPENAI_API_KEY\n")

    print("Se creó el archivo .env con las variables iniciales.")
    # Vuelve a cargar las variables de entorno después de crear el archivo
    load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY") or "MY_OPENAI_API_KEY"

# Importamos librerias

In [ ]:
from operator import itemgetter

from langchain_community.vectorstores.faiss import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.callbacks import get_openai_callback

# Modelo LLM

In [ ]:
# Load vectorstore from local storage

embeddings = OpenAIEmbeddings()

persisted_vectorstore = FAISS.load_local("scripts/faiss_index", embeddings)


In [ ]:
retriever = persisted_vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

System: Answer according to the language you are typing in the prompt {question}. 
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-4-0125-preview")

In [ ]:
chain = (
    {'context':retriever, "question":RunnablePassthrough()}
    |prompt
    |model
    |StrOutputParser()
)

In [ ]:
with get_openai_callback() as cb:
    salida = chain.invoke("Hola, como estas?")
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${format(cb.total_cost, '.20f')}")
    print(salida)